In [1]:
import time
import random
import numpy as np
import pandas as pd
import sys
import pickle
import h5py
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '2'
import torch
import learn2learn as l2l
from tqdm import tqdm
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from torch.optim.lr_scheduler import StepLR, CosineAnnealingLR
from functions.fftc import fft2c_new as fft2c
from functions.fftc import ifft2c_new as ifft2c
from functions.math import complex_abs, complex_mul, complex_conj
# The corase reconstruction is the rss of the zerofilled multi-coil kspaces
# after inverse FT.
from functions.data.transforms import UnetDataTransform_sens_TTT, complex_center_crop, center_crop_to_smallest
# Import a torch.utils.data.Dataset class that takes a list of data examples, a path to those examples
# a data transform and outputs a torch dataset.
from functions.data.mri_dataset import SliceDataset
# Unet architecture as nn.Module
from functions.models.unet import Unet
# Function that returns a MaskFunc object either for generatig random or equispaced masks
from functions.data.subsample import create_mask_for_mask_type
# Implementation of SSIMLoss
from functions.training.losses import SSIMLoss
from functions.helper import evaluate2c_imagepair
### after you install bart 0.7.00 from https://mrirecon.github.io/bart/, import it as follows
sys.path.insert(0,'/cheng/bart-0.7.00/python/')
os.environ['TOOLBOX_PATH'] = "/cheng/bart-0.7.00/"
import bart


plt.rcParams.update({"text.usetex": True, "font.family": "serif", "font.serif": ["Computer Modern Roman"]})

colors = ['b','r','k','g','m','c','tab:brown','tab:orange','tab:pink','tab:gray','tab:olive','tab:purple']

markers = ["v","o","^","1","*",">","d","<","s","P","X"]
FONTSIZE = 22

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# seed
SEED = 1
random.seed(SEED)
np.random.seed(SEED)
torch.cuda.manual_seed(SEED)
torch.manual_seed(SEED)


### Load the data

In [3]:
# data path
# path_test = '/cheng/metaMRI/metaMRI/data_dict/E11.1/Q/brain_test_AXT1POST_Skyra_5-8.yaml'
# path_test_sensmaps = '/cheng/metaMRI/metaMRI/data_dict/E11.1/Q/sensmap_test/'
path_test = '/cheng/metaMRI/metaMRI/data_dict/TTT_paper/TTT_brain_val.yaml'
path_test_sensmaps = '/cheng/metaMRI/metaMRI/data_dict/TTT_paper/sensmap_brain_val/'

# mask function and data transform
mask_function = create_mask_for_mask_type(mask_type_str = 'random', self_sup = False, 
                    center_fraction = 0.08, acceleration = 4.0, acceleration_total = 3.0)

data_transform = UnetDataTransform_sens_TTT('multicoil', mask_func = mask_function, use_seed=True, mode='adapt')

# training dataset and data loader
testset = SliceDataset(dataset = path_test, path_to_dataset='', 
                path_to_sensmaps = path_test_sensmaps, provide_senmaps=True, 
                challenge="multicoil", transform = data_transform, use_dataset_cache=True)

test_dataloader = torch.utils.data.DataLoader(dataset = testset, batch_size = 1, 
                shuffle = False, generator = torch.Generator().manual_seed(1), pin_memory = False)


### without TTT

In [10]:
checkpoint_path = '/cheng/metaMRI/metaMRI/save/E11.2_sup(l1_CA-1e-3-4_P)_T300_120epoch/E11.2_sup(l1_CA-1e-3-4_P)_T300_120epoch_E71_best.pth'
# '/cheng/metaMRI/metaMRI/save/E11.2_sup(l1_CA-1e-3-4_P)_T300_120epoch/E11.2_sup(l1_CA-1e-3-4_P)_T300_120epoch_E71_best.pth'
# '/cheng/metaMRI/metaMRI/save/E11.2_sup(l1_CA-1e-3-4_Q)_T300_1200epoch/E11.2_sup(l1_CA-1e-3-4_Q)_T300_1200epoch_E98_best.pth'
model = Unet(in_chans=2, out_chans=2, chans=32, num_pool_layers=4, drop_prob=0.0)
model.load_state_dict(torch.load(checkpoint_path))
model = model.to(device)

In [11]:
loss_l1_history_=[]
loss_ssim_history_=[]
ssim_fct = SSIMLoss()
l1_loss = torch.nn.L1Loss(reduction='sum')

for iter, batch in tqdm(enumerate(test_dataloader)): 
    input_image, target_image, ground_truth_image, mean, std, fname, slice_num, input_kspace, input_mask, target_kspace, target_mask, sens_maps, binary_background_mask = batch
    input_image = input_image.to(device)
    target_image = target_image.to(device)
    input_kspace = input_kspace.to(device)
    input_mask = input_mask.to(device)
    sens_maps = sens_maps.to(device)
    std = std.to(device)
    mean = mean.to(device)
    ground_truth_image = ground_truth_image.to(device)
    binary_background_mask = binary_background_mask.to(device)

    model_output = model(input_image)
    model_output = model_output * std + mean
    # supervised loss [x, fθ(A†y)]
    # L1
    loss_l1 = (l1_loss(model_output, target_image) / torch.sum(torch.abs(target_image))).item()
    # SSIM = 1 - loss
    output_image_1c = complex_abs(torch.moveaxis(model_output , 1, -1 ))
    loss_ssim = 1 - ssim_fct(output_image_1c, ground_truth_image, data_range = ground_truth_image.max().unsqueeze(0)).item()
    #print('Test SSIM loss: ',loss_ssim)

    loss_l1_history_.append(loss_l1)
    loss_ssim_history_.append(loss_ssim)

200it [00:26,  7.51it/s]


In [12]:
print("Testing average L1 loss: ", sum(loss_l1_history_) / len(loss_l1_history_))
print("Testing average SSIM loss: ", sum(loss_ssim_history_) / len(loss_ssim_history_))

Testing average L1 loss:  0.17661301501095295
Testing average SSIM loss:  0.924997610449791


### Test-time training

In [4]:
ssim_fct = SSIMLoss()
l1_loss = torch.nn.L1Loss(reduction='sum')

def TTT(model, TTT_epoch, adapt_lr): 
    optimizer = torch.optim.Adam(model.parameters(),lr=adapt_lr)
    
    loss_l1_history = []
    loss_ssim_history = []
    self_loss_history = []

    for iteration in range(TTT_epoch):
        # fθ(A†y)
        model_output = model(input_image)
        model_output = model_output * std + mean
        # supervised loss [x, fθ(A†y)]
        # L1
        loss_l1 = (l1_loss(model_output, target_image) / torch.sum(torch.abs(target_image))).item()
        #print('Test L1 loss: ',loss_l1)
        loss_l1_history.append(loss_l1)
        # SSIM = 1 - loss
        output_image_1c = complex_abs(torch.moveaxis(model_output , 1, -1 ))
        loss_ssim = 1 - ssim_fct(output_image_1c, ground_truth_image, data_range = ground_truth_image.max().unsqueeze(0)).item()
        #print('Test SSIM loss: ',loss_ssim)
        loss_ssim_history.append(loss_ssim)

        # self-supervised loss
        # fθ(A†y)
        model_output = torch.moveaxis(model_output, 1, -1 )
        # S fθ(A†y)
        output_sens_image = complex_mul(model_output, sens_maps)
        # FS fθ(A†y)
        Fimg = fft2c(output_sens_image)
        # MFS fθ(A†y) = A fθ(A†y)
        Fimg_forward = Fimg * input_mask
        # consistency loss [y, Afθ(A†y)]
        loss_self = l1_loss(Fimg_forward, input_kspace) / torch.sum(torch.abs(input_kspace))

        optimizer.zero_grad()
        loss_self.backward()
        optimizer.step()
        #train_loss += loss.item()
        #print('TTT loss: ',loss_self.item())
        self_loss_history.append(loss_self.item())

    return loss_l1_history, loss_ssim_history, self_loss_history

In [6]:
checkpoint_path_self = '/cheng/metaMRI/metaMRI/save/E11.3_joint(l1_CA-1e-3-4_Q)_T300_120epoch/E11.3_joint(l1_CA-1e-3-4_Q)_T300_120epoch_E111_best.pth'
# '/cheng/metaMRI/metaMRI/save/E11.2_joint(l1_CA-1e-3-4_P)_T300_120epoch/E11.2_joint(l1_CA-1e-3-4_P)_T300_120epoch_E75_best.pth'
# '/cheng/metaMRI/metaMRI/save/E11.3_joint(l1_CA-1e-3-4_P)_T300_120epoch/E11.3_joint(l1_CA-1e-3-4_P)_T300_120epoch_E66_best.pth'

model_self = Unet(in_chans=2, out_chans=2, chans=32, num_pool_layers=4, drop_prob=0.0)
model_self.load_state_dict(torch.load(checkpoint_path_self))
model_self = model_self.to(device)

best_loss_l1_history=[]
best_loss_l1_index_history=[]
best_loss_ssim_history=[]
best_loss_ssim_index_history=[]

for iter, batch in tqdm(enumerate(test_dataloader)): 
    input_image, target_image, ground_truth_image, mean, std, fname, slice_num, input_kspace, input_mask, target_kspace, target_mask, sens_maps, binary_background_mask = batch
    input_image = input_image.to(device)
    target_image = target_image.to(device)
    input_kspace = input_kspace.to(device)
    input_mask = input_mask.to(device)
    sens_maps = sens_maps.to(device)
    std = std.to(device)
    mean = mean.to(device)
    ground_truth_image = ground_truth_image.to(device)
    binary_background_mask = binary_background_mask.to(device)

    # model re-init
    model_self.load_state_dict(torch.load(checkpoint_path_self))

    # each data point TTT
    loss_l1_history, loss_ssim_history, self_loss_history = TTT(model_self, TTT_epoch=500, adapt_lr=0.0001)
    best_loss_l1 = min(loss_l1_history)
    print('best L1: ', best_loss_l1)
    best_loss_l1_epoch = np.argmin(loss_l1_history)
    print('best L1 epoch: ', best_loss_l1_epoch)
    best_loss_ssim = max(loss_ssim_history)
    print('best SSIM:', best_loss_ssim)
    best_loss_ssim_epoch = np.argmax(loss_ssim_history)
    print('best SSIM epoch: ', best_loss_ssim_epoch)

    best_loss_l1_index_history.append(best_loss_l1_epoch)
    best_loss_ssim_index_history.append(best_loss_ssim_epoch)
    best_loss_l1_history.append(best_loss_l1)
    best_loss_ssim_history.append(best_loss_ssim)


print("Testing average L1 loss: ", sum(best_loss_l1_history) / len(best_loss_l1_history))
print("Testing average L1 loss epoch: ", sum(best_loss_l1_index_history) / len(best_loss_l1_index_history))
print("Testing average SSIM loss: ", sum(best_loss_ssim_history) / len(best_loss_ssim_history))
print("Testing average SSIM loss epoch: ", sum(best_loss_ssim_index_history) / len(best_loss_ssim_index_history))

1it [00:20, 20.67s/it]

best L1:  0.10670609772205353
best L1 epoch:  179
best SSIM: 0.9699586629867554
best SSIM epoch:  162


2it [00:40, 20.44s/it]

best L1:  0.09667706489562988
best L1 epoch:  134
best SSIM: 0.9734474420547485
best SSIM epoch:  137


3it [01:01, 20.39s/it]

...

best L1:  0.11347907036542892
best L1 epoch:  204
best SSIM: 0.9617080688476562
best SSIM epoch:  175


199it [1:10:17, 21.40s/it]

best L1:  0.11716939508914948
best L1 epoch:  123
best SSIM: 0.9584152698516846
best SSIM epoch:  50


200it [1:10:39, 21.20s/it]

best L1:  0.11932849884033203
best L1 epoch:  262
best SSIM: 0.9563634991645813
best SSIM epoch:  29
Testing average L1 loss:  0.11496226418763399
Testing average L1 loss epoch:  176.82
Testing average SSIM loss:  0.957665029168129
Testing average SSIM loss epoch:  124.81


Meta

In [4]:
checkpoint_path_meta =  '/cheng/metaMRI/metaMRI/save/E11.2_maml(l1_CA-1e-3-4_Q)_T300_200epoch/E11.2_maml(l1_CA-1e-3-4_Q)_T300_200epoch_E98_best.pth'
# '/cheng/metaMRI/metaMRI/save/E11.1_maml(L1-5adapt)_T300_200epoch/E11.1_maml(L1-5adapt)_T300_200epoch_E79_best.pth'
# '/cheng/metaMRI/metaMRI/save/E11.3_maml(l1_CA-1e-3-4_P)_T300_200epoch/E11.3_maml(l1_CA-1e-3-4_P)_T300_200epoch_E79_best.pth'
# '/cheng/metaMRI/metaMRI/save/E11.3_maml(l1_CA-1e-3-4_P)_adapt10_T300_200epoch/E11.3_maml(l1_CA-1e-3-4_P)_adapt10_T300_200epoch_E92_best.pth'

model_meta = Unet(in_chans=2, out_chans=2, chans=32, num_pool_layers=4, drop_prob=0.0)
model_meta.load_state_dict(torch.load(checkpoint_path_meta))
model_meta = model_meta.to(device)


best_loss_l1_history=[]
best_loss_l1_index_history=[]
best_loss_ssim_history=[]
best_loss_ssim_index_history=[]

for iter, batch in tqdm(enumerate(test_dataloader)): 
    input_image, target_image, ground_truth_image, mean, std, fname, slice_num, input_kspace, input_mask, target_kspace, target_mask, sens_maps, binary_background_mask = batch
    input_image = input_image.to(device)
    target_image = target_image.to(device)
    input_kspace = input_kspace.to(device)
    input_mask = input_mask.to(device)
    sens_maps = sens_maps.to(device)
    std = std.to(device)
    mean = mean.to(device)
    ground_truth_image = ground_truth_image.to(device)
    binary_background_mask = binary_background_mask.to(device)

    # model re-init
    model_meta.load_state_dict(torch.load(checkpoint_path_meta))

    # each data point TTT
    loss_l1_history, loss_ssim_history, self_loss_history = TTT(model_meta, TTT_epoch=500, adapt_lr=0.0001)
    best_loss_l1 = min(loss_l1_history)
    print('best L1: ', best_loss_l1)
    best_loss_l1_epoch = np.argmin(loss_l1_history)
    print('best L1 epoch: ', best_loss_l1_epoch)
    best_loss_ssim = max(loss_ssim_history)
    print('best SSIM:', best_loss_ssim)
    best_loss_ssim_epoch = np.argmax(loss_ssim_history)
    print('best SSIM epoch: ', best_loss_ssim_epoch)

    best_loss_l1_index_history.append(best_loss_l1_epoch)
    best_loss_ssim_index_history.append(best_loss_ssim_epoch)
    best_loss_l1_history.append(best_loss_l1)
    best_loss_ssim_history.append(best_loss_ssim)


print("Testing average L1 loss: ", sum(best_loss_l1_history) / len(best_loss_l1_history))
print("Testing average L1 loss epoch: ", sum(best_loss_l1_index_history) / len(best_loss_l1_index_history))
print("Testing average SSIM loss: ", sum(best_loss_ssim_history) / len(best_loss_ssim_history))
print("Testing average SSIM loss epoch: ", sum(best_loss_ssim_index_history) / len(best_loss_ssim_index_history))

1it [00:24, 24.15s/it]

best L1:  0.09542027115821838
best L1 epoch:  271
best SSIM: 0.9753828048706055
best SSIM epoch:  240


2it [00:44, 21.90s/it]

best L1:  0.08968939632177353
best L1 epoch:  293
best SSIM: 0.9772889614105225
best SSIM epoch:  283


3it [01:04, 21.25s/it]

...

best L1:  0.10394541174173355
best L1 epoch:  175
best SSIM: 0.9665471315383911
best SSIM epoch:  116


199it [1:14:26, 22.97s/it]

best L1:  0.10637722164392471
best L1 epoch:  92
best SSIM: 0.9646404385566711
best SSIM epoch:  87


200it [1:14:49, 22.45s/it]

best L1:  0.11155816912651062
best L1 epoch:  196
best SSIM: 0.9626089930534363
best SSIM epoch:  84
Testing average L1 loss:  0.10637188885360956
Testing average L1 loss epoch:  211.95
Testing average SSIM loss:  0.9640017288923264
Testing average SSIM loss epoch:  159.63
